In [ ]:
import sqlite3
from contextlib import closing
from datetime import datetime

import streamlit as st

DB_PATH = "lab_inventory.db"

# -----------------------------
# DB helpers
# -----------------------------
def init_db():
    with closing(sqlite3.connect(DB_PATH)) as conn, conn, closing(conn.cursor()) as cur:
        cur.execute("""
        CREATE TABLE IF NOT EXISTS chemicals (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT UNIQUE NOT NULL,
            amount REAL NOT NULL DEFAULT 0,
            unit TEXT NOT NULL DEFAULT 'g',
            location TEXT DEFAULT '',
            notes TEXT DEFAULT ''
        );""")
        cur.execute("""
        CREATE TABLE IF NOT EXISTS requests (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            chem_id INTEGER NOT NULL,
            requester_email TEXT NOT NULL,
            quantity REAL NOT NULL,
            status TEXT NOT NULL DEFAULT 'pending',
            created_at TEXT NOT NULL,
            FOREIGN KEY (chem_id) REFERENCES chemicals(id)
        );""")

def get_conn():
    return sqlite3.connect(DB_PATH, check_same_thread=False)

def list_chemicals(search=""):
    with closing(get_conn()) as conn, closing(conn.cursor()) as cur:
        if search:
            cur.execute("""
                SELECT id, name, amount, unit, location FROM chemicals
                WHERE LOWER(name) LIKE ? ORDER BY name ASC
            """, (f"%{search.lower()}%",))
        else:
            cur.execute("SELECT id, name, amount, unit, location FROM chemicals ORDER BY name ASC")
        rows = cur.fetchall()
    return rows

def add_chemical(name, amount, unit, location, notes=""):
    with closing(get_conn()) as conn, conn, closing(conn.cursor()) as cur:
        cur.execute("""
            INSERT OR IGNORE INTO chemicals (name, amount, unit, location, notes)
            VALUES (?, ?, ?, ?, ?)
        """, (name.strip(), amount, unit.strip(), location.strip(), notes.strip()))

def update_stock(chem_id, delta_amount):
    with closing(get_conn()) as conn, conn, closing(conn.cursor()) as cur:
        cur.execute("UPDATE chemicals SET amount = amount + ? WHERE id = ?", (delta_amount, chem_id))

def add_request(chem_id, email, qty):
    now = datetime.utcnow().isoformat(timespec="seconds")
    with closing(get_conn()) as conn, conn, closing(conn.cursor()) as cur:
        cur.execute("""
            INSERT INTO requests (chem_id, requester_email, quantity, status, created_at)
            VALUES (?, ?, ?, 'pending', ?)
        """, (chem_id, email.strip(), qty, now))

def list_requests(status=None):
    with closing(get_conn()) as conn, closing(conn.cursor()) as cur:
        if status:
            cur.execute("""
            SELECT r.id, c.name, r.quantity, r.requester_email, r.status, r.created_at
            FROM requests r JOIN chemicals c ON r.chem_id = c.id
            WHERE r.status = ?
            ORDER BY r.id DESC
            """, (status,))
        else:
            cur.execute("""
            SELECT r.id, c.name, r.quantity, r.requester_email, r.status, r.created_at
            FROM requests r JOIN chemicals c ON r.chem_id = c.id
            ORDER BY r.id DESC
            """)
        return cur.fetchall()

def set_request_status(req_id, new_status):
    with closing(get_conn()) as conn, conn, closing(conn.cursor()) as cur:
        cur.execute("UPDATE requests SET status = ? WHERE id = ?", (new_status, req_id))

def get_chemical(chem_id):
    with closing(get_conn()) as conn, closing(conn.cursor()) as cur:
        cur.execute("SELECT id, name, amount, unit FROM chemicals WHERE id = ?", (chem_id,))
        return cur.fetchone()

# -----------------------------
# App UI
# -----------------------------
st.set_page_config(page_title="Lab Chemicals", page_icon="🧪", layout="wide")

# Replace this with st.secrets for real deployments
ADMIN_PASSWORD = st.secrets.get("ADMIN_PASSWORD", "change-me")

init_db()

st.title("🧪 Lab Chemical Inventory")

tabs = st.tabs(["Search & Request", "Admin"])

# -----------------------------
# Tab 1: Search & Request
# -----------------------------
with tabs[0]:
    st.subheader("Search inventory")
    q = st.text_input("Search by chemical name", placeholder="e.g., acetone, ethanol, NaCl")
    data = list_chemicals(q)

    if data:
        # Render a simple table
        st.write("#### Available chemicals")
        st.dataframe(
            [{"ID": r[0], "Chemical": r[1], "Amount": r[2], "Unit": r[3], "Location": r[4]} for r in data],
            use_container_width=True,
        )
    else:
        st.info("No chemicals match your search.")

    st.markdown("---")
    st.subheader("Request a chemical")

    chem_options = {f"{name} ({amount} {unit})": cid for cid, name, amount, unit, _loc in data} or {}

    if not chem_options:
        st.warning("No chemicals to request yet. Please ask an admin to add items.")
    else:
        with st.form("request_form"):
            chosen = st.selectbox("Choose chemical", options=list(chem_options.keys()))
            qty = st.number_input("Quantity needed", min_value=0.0, step=0.1, format="%.3f")
            email = st.text_input("Your email")
            submitted = st.form_submit_button("Submit request")
        if submitted:
            chem_id = chem_options[chosen]
            if qty <= 0:
                st.error("Quantity must be > 0.")
            elif "@" not in email:
                st.error("Please enter a valid email.")
            else:
                add_request(chem_id, email, qty)
                st.success("Request submitted! You’ll be notified by the lab admin.")

# -----------------------------
# Tab 2: Admin
# -----------------------------
with tabs[1]:
    st.subheader("Admin")
    pw = st.text_input("Admin password", type="password")
    if pw != ADMIN_PASSWORD:
        st.warning("Enter the correct password to manage inventory.")
        st.stop()

    st.success("Admin mode enabled.")

    # Add new chemical
    with st.expander("➕ Add a new chemical"):
        with st.form("add_chem"):
            col1, col2, col3 = st.columns(3)
            with col1:
                name = st.text_input("Chemical name")
                unit = st.text_input("Unit", value="g")
            with col2:
                amount = st.number_input("Initial amount", min_value=0.0, step=0.1)
            with col3:
                location = st.text_input("Location", value="")
            notes = st.text_area("Notes (optional)", height=80)
            add_btn = st.form_submit_button("Add")
        if add_btn:
            if not name.strip():
                st.error("Name is required.")
            else:
                add_chemical(name, amount, unit, location, notes)
                st.success(f"Added/updated “{name}”. Refresh the table below to see it.")

    # Adjust stock
    with st.expander("🔧 Adjust stock"):
        all_chems = list_chemicals("")
        if not all_chems:
            st.info("No chemicals yet.")
        else:
            label_to_id = {f"{n} ({a} {u}) [ID:{cid}]": cid for cid, n, a, u, _ in all_chems}
            sel = st.selectbox("Select chemical", list(label_to_id.keys()))
            delta = st.number_input("Change in amount (use negative to reduce)", step=0.1, format="%.3f")
            if st.button("Apply change"):
                update_stock(label_to_id[sel], delta)
                st.success("Stock updated.")

    st.markdown("### Pending requests")
    reqs = list_requests(status="pending")
    if reqs:
        for rid, cname, qty, remail, status, created in reqs:
            with st.container(border=True):
                st.write(f"**[{rid}] {cname}** — requested: **{qty}**")
                st.write(f"Requester: {remail} • Created: {created} • Status: {status}")
                c1, c2, c3 = st.columns(3)
                with c1:
                    if st.button("Approve", key=f"approve_{rid}"):
                        # (Optional) decrease stock on approval:
                        # find chem id from request


2025-08-23 18:30:28.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 18:30:28.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 18:30:28.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 18:30:28.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 18:30:28.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 18:30:28.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 18:30:28.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 18:30:28.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar